## Machine Learning en Spark (Pipeline)

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

import pandas as pd

spark = SparkSession.builder.appName('spark-ml').getOrCreate()
sc = spark.sparkContext

21/12/10 19:10:35 WARN Utils: Your hostname, jesus-Aspire-A514-52 resolves to a loopback address: 127.0.1.1; using 192.168.1.54 instead (on interface wlp2s0)
21/12/10 19:10:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/10 19:10:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/10 19:10:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
print("Running Spark Version %s" % (sc.version))

Running Spark Version 3.2.0


In [2]:
df_cars = spark.read.option("header",True).option("inferSchema",True).csv("car_milage.csv")
df_cars = df_cars.na.drop()

In [3]:
# Features (Modelamos mpg a partir del resto de datos)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline

# Prepare train data
train_df, test_df = df_cars.randomSplit([0.8,0.2], seed=42)

assembler = VectorAssembler(
    inputCols=["displacement","hp","torque","CRatio","RARatio","CarbBarrells","NoOfSpeed","length","width","weight","automatic"],
    outputCol="features")

lr = LinearRegression(labelCol="mpg", maxIter=1000, regParam=0.3, elasticNetParam=0.8)

pipeline = Pipeline(stages=[assembler, lr])

model = pipeline.fit(train_df)


In [17]:
# Predict model
predictions = model.transform(test_df)

# Evaluate
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="mpg",predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)

print("mse = %g" % mse)


mse = 17.0551


In [18]:
# Save model
model.write().overwrite().save("models/lr")

In [19]:
model_data = spark.read.parquet("models/lr/stages/1_LinearRegression_c5f28b1349d0/data/")
model_data.show()

AnalysisException: Path does not exist: file:/home/jesus/universidad/Infraestructuras_Computacionales/practicas/TP2/spark-ml/models/lr/stages/1_LinearRegression_c5f28b1349d0/data

In [ ]:
# Load model
from pyspark.ml import PipelineModel
modelLoaded = PipelineModel.load("models/lr")
new_predictions = modelLoaded.transform(test_df)
new_predictions.show()

+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+--------------------+------------------+
|  mpg|displacement| hp|torque|CRatio|RARatio|CarbBarrells|NoOfSpeed|length|width|weight|automatic|            features|        prediction|
+-----+------------+---+------+------+-------+------------+---------+------+-----+------+---------+--------------------+------------------+
|13.27|       351.0|148|   243|   8.0|   3.26|           2|        3| 216.1| 78.5|  4715|        1|[351.0,148.0,243....|16.320539906777142|
|13.27|       460.0|223|   366|   8.0|    3.0|           4|        3| 228.0| 79.8|  5430|        1|[460.0,223.0,366....|12.082968096851397|
|13.77|       360.0|195|   295|  8.25|   3.15|           4|        3| 209.3| 77.4|  4215|        1|[360.0,195.0,295....|16.939044397471363|
|18.25|       351.0|143|   255|   8.0|    3.0|           2|        3| 199.9| 74.0|  3890|        1|[351.0,143.0,255....|17.362305794523685|
|19.73|       318.0|